<a href="https://colab.research.google.com/github/YvYh/FluxWeb/blob/main/Model_Controverses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **BQ**: Get/Load Data

In [3]:
from google.cloud import bigquery
json_path = 'poc-bigdata.json'
bigquery_client = bigquery.Client.from_service_account_json(json_path)

In [4]:
def get_bq_data(query):
  query_job = bigquery_client.query(query)
  rows = query_job.result()
  data = rows.to_dataframe()
  return data

In [5]:
def bq_load_df(name, df):
    dataset_ref = bigquery_client.dataset('FluxWeb_Prediction')
    table_ref = dataset_ref.table(name)
    
    
    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND
    if name == 'Controverses_bert':
      job_config.schema = get_bigquery_schema()
    else:
      job_config.autodetect=True
 
    load_job = bigquery_client.load_table_from_dataframe(
        df,
        table_ref,
        job_config=job_config)
 
    assert load_job.job_type == 'load'
 
    load_job.result()  # Waits for table load to complete.
 
    assert load_job.state == 'DONE'
    print('table {} load {} data.'.format(name, len(df)))

# Bert


In [ ]:
!pip install -q tensorflow-text
!pip install -q tf-models-official

     |████████████████████████████████| 3.4MB 5.9MB/s 
     |████████████████████████████████| 1.1MB 5.8MB/s 
     |████████████████████████████████| 174kB 47.2MB/s 
     |████████████████████████████████| 358kB 41.5MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 37.6MB 96kB/s 
     |████████████████████████████████| 706kB 40.3MB/s 
     |████████████████████████████████| 102kB 9.9MB/s 
     |████████████████████████████████| 645kB 38.1MB/s 
     |████████████████████████████████| 1.2MB 34.7MB/s 


In [ ]:
!pip install --upgrade scikit-learn

     |████████████████████████████████| 22.3MB 2.0MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
tf.get_logger().setLevel('ERROR')

import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3'
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3'
 
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [ ]:
def bert_vectorize(text_test):
    
    text_preprocessed = bert_preprocess_model([text_test])
    bert_results = bert_model(text_preprocessed)
    vect = bert_results["pooled_output"][0,:]
    return vect.numpy()
    #return bert_vectorize

In [ ]:
from IPython.display import HTML, display
import time
 
def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

In [ ]:
q_news = """SELECT distinct id_news,News, NumControverse 
FROM `poc-bigdata.FluxWeb_Prediction.News_test` 
Order by id_news desc
"""
news = get_bq_data(q_news)
news.head()

,id_news,News,NumControverse
0,N957069,Carrefour mise un milliard sur les anciens act...,3265
1,N957069,Carrefour mise un milliard sur les anciens act...,1689
2,N957069,Carrefour mise un milliard sur les anciens act...,3006
3,N957069,Carrefour mise un milliard sur les anciens act...,3412
4,N957069,Carrefour mise un milliard sur les anciens act...,2722


In [ ]:
import pickle
svd=pickle.load(open('svd120.pickle', 'rb'))

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator TruncatedSVD from version 0.23.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [ ]:
cosine_similarity(svd.transform([news_bert]),svd.transform(controverses.vect_c.to_list()))[0]

array([0.98004867, 0.8753628 , 0.90957654])

In [ ]:
nb = len(news.id_news.unique())
print(str(nb))
i = 0
out = display(progress(0, nb), display_id=True)
for idn in news.id_news.unique():

  news_text = news[news['id_news']==idn]['News'].iloc[0]
  news_bert = bert_vectorize(news_text)
  nums = news[news['id_news']==idn]['NumControverse']
  #print(i/nb)

  result = pd.DataFrame()
  for idc in nums:
    q_vectC = "SELECT NumControverse,Poids,[Embed_"+", Embed_".join([str(x) for x in range(768)])+"] vect_c FROM `poc-bigdata.FluxWeb_Prediction.Controverses_bert` WHERE NumControverse={}".format(str(idc))
    controverses = get_bq_data(q_vectC)
    cos = cosine_similarity([news_bert],controverses.vect_c.to_list())[0]
    cos120 = cosine_similarity(svd.transform([news_bert]),svd.transform(controverses.vect_c.to_list()))[0]
    s = controverses.Poids.sum()
    poids = [x/s for x in controverses.Poids]
    result = result.append({'id_news':idn,
                            'numControverse':idc,
                            'bert_score':sum(cos*poids),
                            'bert_svd120_score':sum(cos120*poids)}
                            ,ignore_index=True)
  bq_load_df_in_gcs('Controverse_News_Score', result)
  out.update(progress(i, nb))
  i += 1

# **TensorFlow**:  TextVectorization

### Get Input Data

In [ ]:
import numpy as np
import pandas as pd

In [8]:
q = """SELECT DISTINCT NumControverse as label, Titre as Text
FROM `poc-bigdata.FluxWeb_Prediction.Controverses_prd`
UNION ALL 
SELECT DISTINCT NumControverse as label, Commentaire as Text
FROM `poc-bigdata.FluxWeb_Prediction.Controverses_prd`
WHERE Commentaire is not null and length(Commentaire)>5
UNION ALL 
SELECT DISTINCT NumControverse as label, Informations as Text
FROM `poc-bigdata.FluxWeb_Prediction.Controverses_prd`
WHERE Informations is not null
"""
controverses = get_bq_data(q)

In [47]:
#controverses['Text_clean']=controverses.Text.apply(text_preprossing)
controverses = controverses[['label','Text']]

In [62]:
print(len(controverses))
print(len(controverses.label.unique()))
controverses.head()

3751
1408


,label,Text
0,2866,Révélation d'un scandale comptable survenu en ...
1,389,EPR de Flamanville: malfaçons
2,2543,Condamnation suite à des déversements dans des...
3,3506,Poursuites aux Etats-Unis en lien avec les émi...
4,3016,Incendie dans une usine de Lubrizol à Rouen


In [ ]:
!pip install -U spacy
!python -m spacy download fr_core_news_lg

In [ ]:
import spacy
nlp = spacy.load("fr_core_news_lg")

In [107]:
def text_preprossing(text):
    tokens = nlp(text)
    clean = []
    for token in tokens:
        if not any([token.is_digit, token.is_punct, token.is_currency,
               token.like_url, token.like_num, token.like_email,
               token.is_stop]):
            clean += [token.lemma_]
    #return tf.strings.join(clean, separator=' ')
    return ' '.join(clean)

In [115]:
data = pd.DataFrame()
data['label'] = controverses.label
data['Text'] = controverses.Text.apply(text_preprossing)
data.head()

,label,Text
0,2866,révélation scandale comptable survenir impliqu...
1,389,epr Flamanville malfaçon
2,2543,condamnation suite déversement rivière
3,3506,poursuite Etats-Unis lien émission no niveau c...
4,3016,incendie usine Lubrizol Rouen


In [118]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

2400 train examples
600 validation examples
751 test examples


In [155]:
val.head()

,label,Text
1212,69,procédure cours cadre fait avérer corruption
3551,3178,ancien avocat filiale conglomérat Danaher Corp...
1040,3370,reprise opération Carnival Etats-Unis
2858,2604,syndicat approuver octobre accord proposer pdg...
69,1407,rapport critique institution financier finance...


In [120]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  #dataframe = dataframe.copy()
  #labels = dataframe.pop('label')
  ds = tf.data.Dataset.from_tensor_slices((dataframe.Text.values, dataframe.label.values))
  #ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [121]:
batch_size = 32 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)
train_ds

<BatchDataset shapes: ((None,), (None,)), types: (tf.string, tf.int64)>

In [122]:
for text_batch, label_batch in train_ds.take(1):
  for i in range(5):
    print(label_batch[i].numpy(), text_batch.numpy()[i])

3319 b'enqu\xc3\xaate conduire ICIJ m\xc3\xa9dia montre grand banque poreux blanchiment argent base suspiciou activity report SAR transmettre banque autorit\xc3\xa9 lutte antiblanchiment am\xc3\xa9ricain fincen rapport ultraconfidentiels repr\xc3\xa9senter total mds$ transaction suspect r\xc3\xa9aliser montrer banque circuler passivement travers compte bancaire personne soci\xc3\xa9t\xc3\xa9 identifier argent susceptible relever blanchiment aucun banque solliciter souhaiter r\xc3\xa9agir fincen proposer septembre vaste r\xc3\xa9forme cens\xc3\xa9 am\xc3\xa9liorer efficacit\xc3\xa9 lutte anti-blanchiment renforcer obligation d\xc3\xa9claration banque   '
3460 b'critique politique \xc3\xa9conomique chinois fondateur cons\xc3\xa9quence'
1330 b'ong n\xc3\xa9erlandais milieudefensie enqu\xc3\xaater fournisseur huile palme Neste neste acheter mt distillat acide gras palme pfad d\xc3\xa9chet production huile palme principalement malaisie indon\xc3\xa9sie ong accuse Neste destruction for\xc3\x

In [123]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

-------  
### Text preprocessing

In [2]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [124]:
vocab_size = 10000
sequence_length = 600

# Use the text vectorization layer to normalize, split, and map strings to
# integers. Set output_sequence_length length to pad all samples to same length.
vectorize_layer = TextVectorization(
    #standardize= text_preprossing,
    standardize='lower_and_strip_punctuation',
    max_tokens=vocab_size,
    output_mode='tf-idf',
    #output_sequence_length=sequence_length
    )

In [ ]:
text_list = train.Text.values.tolist()

In [126]:
vectorize_layer.adapt(text_list)

In [ ]:
vectorize_layer.get_vocabulary()

In [127]:
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
embedding_layer = tf.keras.layers.Embedding(1000, 5)

In [129]:
from tensorflow.keras import Sequential
embedding_dim=1000

model = Sequential([
  vectorize_layer,
  Embedding(vocab_size, embedding_dim, name="embedding"),
  GlobalAveragePooling1D(),
  Dense(16, activation='relu'),
  Dense(1)
])

In [130]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [131]:
model.fit(train_ds, 
          epochs=40, 
          validation_data=val_ds)

Epoch 1/10
75/75 [==============================] - 3s 26ms/step - loss: -779.2183 - accuracy: 0.0000e+00 - val_loss: -1786.3885 - val_accuracy: 0.0000e+00
Epoch 2/10
75/75 [==============================] - 2s 24ms/step - loss: -4096.7046 - accuracy: 0.0000e+00 - val_loss: -7218.8159 - val_accuracy: 0.0000e+00
Epoch 3/10
75/75 [==============================] - 2s 24ms/step - loss: -13170.0039 - accuracy: 0.0000e+00 - val_loss: -20210.8047 - val_accuracy: 0.0000e+00
Epoch 4/10
75/75 [==============================] - 2s 23ms/step - loss: -31861.1738 - accuracy: 0.0000e+00 - val_loss: -44427.7812 - val_accuracy: 0.0000e+00
Epoch 5/10
75/75 [==============================] - 2s 23ms/step - loss: -63834.3125 - accuracy: 0.0000e+00 - val_loss: -83342.9688 - val_accuracy: 0.0000e+00
Epoch 6/10
75/75 [==============================] - 2s 24ms/step - loss: -112532.9219 - accuracy: 0.0000e+00 - val_loss: -140196.9688 - val_accuracy: 0.0000e+00
Epoch 7/10
75/75 [==============================]

In [132]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization_5 (TextVe (None, 10000)             0         
_________________________________________________________________
embedding (Embedding)        (None, 10000, 16)         160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


In [133]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

24/24 [==============================] - 0s 9ms/step - loss: -602432.0625 - accuracy: 0.0000e+00
Accuracy 0.0


#### tfidf model

In [136]:
def text_preprossing2(text):
    tokens = nlp(text)
    clean = []
    for token in tokens:
        if not any([token.is_digit, token.is_punct, token.is_currency,
               token.like_url, token.like_num, token.like_email,
               token.is_stop]):
            clean += [token.lemma_]
    return tf.strings.join(clean, separator=' ')
    #return ' '.join(clean)

In [139]:
vectorize_layer2 = TextVectorization(
    #standardize= text_preprossing,
    standardize='lower_and_strip_punctuation',
    max_tokens=vocab_size,
    output_mode='tf-idf',
    #output_sequence_length=sequence_length
    )

In [140]:
vectorize_layer2.adapt(data.Text.values.tolist())

In [142]:
tfidf = tf.keras.models.Sequential()
tfidf.add(tf.keras.Input(shape=(1,),dtype=tf.string))
tfidf.add(vectorize_layer2)

In [154]:
tfidf.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
tfidf.fit(train.Text.values.tolist(), 
          epochs=40, 
          validation_data=val.Text.values.tolist())

Epoch 1/40


AttributeError: ignored

In [151]:
print(test.Text.values[0])
tfidf.predict([test.Text.values[0]])

US Equal Employment Opportunity commission EEOC intenter action justice tribunal fédéral NY contre Walmart enfreindre loi fédéral laisser employé masculin harceler sexuellement collègue travail prendre mesure cesse lieu mettre fin harcèlement Walmart demander employé harceler défendre contraindre salarié démissionner agence fédéral affirme walmart recevoir plainte sujet comportement harcelant homme partir entreprise prendre mesure efficace faire cesser harcèlement eeoc réclame arriéré salaire dommage intérêt compensatoire dommage intérêt punitif employé concerner mesure redressement prévenir potentiel cas harcèlement sexuel


array([[6.586464, 0.      , 0.      , ..., 0.      , 0.      , 0.      ]],
      dtype=float32)

In [152]:
tfidf.save('tf_idf_model')
#reloaded_model = tf.keras.models.load_model('my_pet_classifier')

INFO:tensorflow:Assets written to: tf_idf_model/assets


In [ ]:
news =''

input_dict = {'Text': tf.convert_to_tensor([news])}
predictions = reloaded_model.predict(input_dict)
prob = tf.nn.sigmoid(predictions[0])

print(
    "This particular pet had a %.1f percent probability "
    "of getting adopted." % (100 * prob)
)